In [ ]:
from mp.irregularPoly import irregularPoly
from mp.irregularPoly import generate_poly
from mp.robot import robot
import random
import utils.vox_mesh as vm
import copy
from run import obj_func
import pickle
import utils.sim_art as simart


# component layer parameters
irrPolyInd = {
    "num_components": 3,
    "evo_type": "component",
    "grammar_iterations": 3,
    "num_points": 10
}

# robot layer parameters
robotInd = {
    "evo_type": "robot",
    "simulation_time": 3600,
    "leg_lim": [1,3],
    "contrl_trials": 1,
    "contrl_trial_gen": 20,
    "contrl_env_friction": 0.9,
    "contrl_env_restitution": 1.0,
    "contrl_env_xlim": [0, 10],
    "contrl_env_ylim": [-5, 5],
    "joint_ulim": 0.2,
    "joint_llim": -0.2,
    "contrl_delta_angle": 0.05,
    "file_path": "data//",
}


def generate_leg(leg, archive):
    square = [[0.11, 0.9], [0.11, 0.11], [0.9, 0.11], [0.9, 0.9]]
    for i in range(0, leg.num_components):
        new_poly, new_edge_in, new_edge_out = generate_poly(leg.dim_x, square)
        leg.poly.append(new_poly)
        leg.edge_idx_in.append(new_edge_in)
        leg.edge_idx_out.append(new_edge_out)
        leg.poly_materials.append(copy.deepcopy(archive[10]))
    print(archive)
    leg.id = vm.id_generator()


def generate_robot(robo, archive):
    robo.num_components = 3
    # pick components from archive
    for i in range(0, robo.num_components):
        idx = random.randint(0, len(archive) - 1)
        robo.parts.append(copy.deepcopy(archive[idx]))
        robo.joints.append([0, 1, 0, 1])
    robo.id = vm.id_generator()


def evaluate():
    random.seed(0)
    friction_values = [0.25, 0.50, 0.75, 1.0]
    restitution_values = [0.25, 0.50, 0.75, 1.0]

    # friction_values = [1.0, 1.0, 1.0, 1.0]
    # restitution_values = [1.0, 1.0, 1.0, 1.0]

    materials = [[i, j] for i in friction_values
                 for j in restitution_values]

    # generate square legs
    leg = irregularPoly(irrPolyInd)
    generate_leg(leg, materials)
    leg.grammar = [[0, 0], [0], [1]]
    leg.axiom = 1
    obj_func(leg)

    # generate robot
    robo = robot(robotInd)
    generate_robot(robo, [leg])
    obj_func(robo)
    robo.fitness, pos = simart.simulate(robo, 2) 

    print(robo.fitness)
    print(pos)
    print("awe")


    alt_robo = copy.deepcopy(robo)
    alt_robo.ctrl_env_friction = 0.05
    alt_robo.ctrl_env_restitution = 0.05
    alt_robo.fitness, pos = simart.simulate(alt_robo, 2)

    # save robot
    with open("data//"+robo.id + "_HFfit" + str(robo.fitness), 'wb') as pickle_file:
        pickle.dump(robo, pickle_file)

    with open("data//"+alt_robo.id + "_LFfit" + str(alt_robo.fitness), 'wb') as pickle_file:
        pickle.dump(alt_robo, pickle_file)
    print(alt_robo.fitness)
    print(pos)

if __name__ == "__main__":
    evaluate()
